In [23]:
import pandas as pd
import numpy as np
import math

weather = pd.read_csv('../data/input/weather.csv')
train = pd.read_csv('../data/input/train.csv')
test = pd.read_csv('../data/input/test.csv')
spray = pd.read_csv('../data/input/spray.csv')

In [24]:
# reducing the list of columns in weather
weather_excluded = ['Depth', 'Water1', 'SnowFall', 'Depart', 'Heat', 'Cool', 'Sunrise', 'Sunset']
weather_keep = [column for column in weather.columns if column not in weather_excluded]
weather = weather[weather_keep]

In [25]:
# dates to date objects
weather.Date = pd.DatetimeIndex(weather.Date)
train.Date = pd.to_datetime(train.Date)
test.Date = pd.to_datetime(test.Date)

In [26]:
# to remove the leading spaces
weather.PrecipTotal = weather.PrecipTotal.str.strip()

#### Looking at daily data, some dates have an "M" or a &"T" in the field. What does this mean?
"M" stands for "Missing". Data for an element will be missing if the primary sensor for that weather element is inoperable (e.g., has an outage) or malfunctioning (e.g., producing errant data) AND any collocated backup sensor is also inoperable or malfunctioning. "T" stand for "Trace". This is a small amount of precipitation that will wet a raingage but is less than the 0.01 inch measuring limit.

In [27]:
weather = weather.replace('T', 0.005)
weather = weather.replace('M', np.nan)

# Train formatting

In [28]:
# Train
train.Date = pd.to_datetime(train.Date)



In [29]:
'''
Formula for calculating compass bearing between two lat/lon tuples.  Credit:  https://gist.github.com/jeromer/2005586
Corrected output error in which returned bearing (in degrees) needed to be subtracted from 360 in order to be correct.  
Validated this change on https://www.sunearthtools.com/tools/distance.php and with manual orienteering using Google 
maps in place of a physical map.

Other than that change at the end of the formula, the code was taken from the gist page linked above.
'''


def compass_bearing(loc1, loc2):
    """
    Calculates the bearing between two points.
    The formulae used is the following:
        θ = atan2(sin(Δlong).cos(lat2),
                  cos(lat1).sin(lat2) − sin(lat1).cos(lat2).cos(Δlong))
    :Parameters:
      - `loc1: The tuple representing the latitude/longitude for the
        first point. Latitude and longitude must be in decimal degrees
      - `loc2: The tuple representing the latitude/longitude for the
        second point. Latitude and longitude must be in decimal degrees
    :Returns:
      The bearing in degrees
    :Returns Type:
      float
    """
    if (type(loc1) != tuple) or (type(loc2) != tuple):
        raise TypeError("Only tuples are supported as arguments")

    lat1 = math.radians(loc1[0])
    lat2 = math.radians(loc2[0])

    diffLong = math.radians(loc1[1] - loc2[1])

    x = math.sin(diffLong) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1)
            * math.cos(lat2) * math.cos(diffLong))

    initial_bearing = math.atan2(x, y)

    # Now we have the initial bearing but math.atan2 return values
    # from -180° to + 180° which is not what we want for a compass bearing
    # The solution is to normalize the initial bearing as shown below
    initial_bearing = math.degrees(initial_bearing)
    compass_bearing = 360 - ((initial_bearing + 360) % 360) ## Mike: this originally returned an incorrect bearing
                         ## corrected by subtracting result from 360

    return compass_bearing

''' 
Calculate distance in KM between two sets of coordinates (lat/lon tuples).  Uses Haversine formula.  Credit to https://gist.github.com/rochacbruno/2883505 and http://www.movable-type.co.uk/scripts/latlong.html

This formula works as-is.  I tested it using Google Maps distance calculator as a validator.
'''

def distance(loc1, loc2):
#     print (loc1, loc2)
    lat1, lon1 = loc1
    lat2, lon2 = loc2
    radius = 6371 # radius of Earth in KM

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

In [30]:
"""
Mapping of each trap to its nearest weather station--compares distance in km between a trap and each airport weather station, and returns the station number ('Station' column) of the nearest weather station.

Weather station info from Kaggle:

Station 1: CHICAGO O'HARE INTERNATIONAL AIRPORT Lat: 41.995 Lon: -87.933 Elev: 662 ft. above sea level
Station 2: CHICAGO MIDWAY INTL ARPT Lat: 41.786 Lon: -87.752 Elev: 612 ft. above sea level

"""

station_coords = {
        1: (41.995,-87.933),
           2: (41.786,-87.752)}

train['Coordinates'] = train[['Latitude', 'Longitude']].apply(tuple, axis=1)

def assign_station(i):
    if distance(station_coords[1], i) < distance(station_coords[2], i):
        return 1
    else:
        return 2

train['Weather_Station'] = train.Coordinates.apply(assign_station)


In [31]:
train.loc[train['Coordinates'] == (41.992478000000006, -87.862994999999998), 'Trap'] = 'T009Alt'

# Back to Weather

In [32]:
weather_types = {
'\+FC': 'TORNADO/WATERSPOUT','FC': 'FUNNEL CLOUD','TS': 'THUNDERSTORM','GR': 'HAIL','RA': 'RAIN',
'DZ': 'DRIZZLE','SN': 'SNOW','SG': 'SNOW GRAINS','GS': 'SMALL HAIL ANDOR SNOW PELLETS','PL': 'ICE PELLETS',
'IC': 'ICE CRYSTALS','FG\+': 'HEAVY FOG','FG': 'FOG','BR': 'MIST','UP': "UNKNOWN PRECIPITATION",'HZ': 'HAZE','FU': 'SMOKE',
'VA': 'VOLCANIC ASH','DU': 'WIDESPREAD DUST','DS': 'DUSTSTORM','PO': 'SAND_DUST WHIRLS',
'SA': 'SAND','SS': 'SANDSTORM','PY': 'SPRAY','SQ': 'SQUALL','DR': 'LOW DRIFTING','SH': 'SHOWER','FZ': 'FREEZING',
'MI': 'SHALLOW','PR': 'PARTIAL','BC': 'PATCHES','BL': 'BLOWING','VC': 'VICINITY'}

for i in weather_types:
    weather[i] = 0
    weather.loc[weather.CodeSum.str.contains(i) == True,i] = 1

weather.drop('CodeSum',axis=1,inplace=True)

In [33]:
weather.head()

,Station,Date,Tmax,Tmin,Tavg,DewPoint,WetBulb,PrecipTotal,StnPressure,SeaLevel,...,MI,TS,SH,RA,\+FC,SA,SG,PO,PL,SN
0,1,2007-05-01,83,50,67,51,56,0.00,29.10,29.82,...,0,0,0,0,0,0,0,0,0,0
1,2,2007-05-01,84,52,68,51,57,0.00,29.18,29.82,...,0,0,0,0,0,0,0,0,0,0
2,1,2007-05-02,59,42,51,42,47,0.00,29.38,30.09,...,0,0,0,0,0,0,0,0,0,0
3,2,2007-05-02,60,43,52,42,47,0.00,29.44,30.08,...,0,0,0,0,0,0,0,0,0,0
4,1,2007-05-03,66,46,56,40,48,0.00,29.39,30.12,...,0,0,0,0,0,0,0,0,0,0


In [38]:
'''Fix this if I have time?'''
# def n_1_weather(df):
#     for i in range(1,7):
#         n_date = '_date-'+str(i)
#         df.loc[:,n_date] = df.Date-pd.DateOffset(i)
#         df = pd.merge(left=df,right=weather,left_on=[n_date,'Station'],right_on=['Date','Station'],suffixes =('',n_date))

In [39]:
# n_1_weather(weather)

In [45]:
n_weather = weather.iloc[12:,:]
for i in range(1,7): # Hard-coded range of the last 6 days
    n_date = "_date-"+str(i)
    n_weather.loc[:,n_date] = n_weather.Date-pd.DateOffset(i)
    n_weather = pd.merge(left=n_weather,right=weather,left_on=[n_date,'Station'],right_on=['Date','Station'],suffixes =('',n_date))

/Users/austinbrian/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [47]:
n_weather.head().T

,0,1,2,3,4
Station,1,2,1,2,1
Date,2007-05-07 00:00:00,2007-05-07 00:00:00,2007-05-08 00:00:00,2007-05-08 00:00:00,2007-05-09 00:00:00
Tmax,83,84,82,80,77
Tmin,47,50,54,60,61
Tavg,65,67,68,70,69
DewPoint,41,39,58,57,59
WetBulb,54,53,62,63,63
PrecipTotal,0.005,0.00,0.00,0.005,0.13
StnPressure,29.38,29.44,29.29,29.36,29.21
SeaLevel,30.12,30.12,30.03,30.02,29.94


# Geography

In [49]:
# Creating de-duped df of traps and coordinates.
traps_master = train[['Trap', 'Coordinates']]
# Drop duplicates
traps_master.drop_duplicates(inplace=True)

/Users/austinbrian/anaconda/lib/python2.7/site-packages/pandas/util/decorators.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return func(*args, **kwargs)


In [50]:
# Create list of column names to use with for loops below
trap_cols = traps_master.Trap.tolist() 

# Create dictionary of Trap IDs and corresponding lat/lon coordinate tuples
trap_dict = traps_master.set_index('Trap')['Coordinates'].to_dict()

In [51]:
# Creates dictionary where key is a Trap name, and the value for each is a list of the distances to every other trap.
distance_dict = {}
for i in trap_cols:
    dist_list = []
    for k in trap_dict:
        dist = distance(trap_dict[k], trap_dict[i])
        dist_list.append(dist)
    distance_dict[i] = dist_list


# Creates dictionary where key is a Trap name, and the value for each is a list of the compass bearings from every other trap.
bearing_dict = {}
for c in trap_cols:
    bearing_list = []
    for q in trap_dict:
        bearing = compass_bearing(trap_dict[q], trap_dict[c])
        bearing_list.append(bearing)
    bearing_dict[c] = bearing_list

In [52]:
# Creates df of relative distances.  To be merged with a left join into train df.
dist_df = pd.DataFrame.from_dict(distance_dict, orient='index')
distance_labels = dist_df.index.tolist()
distance_cols = []
for c in distance_labels:
    label = c + '_dist'
    distance_cols.append(label)
dist_df.columns = [distance_cols] 
dist_df['Trap'] = dist_df.index
# dist_df.to_csv('../data/relative_locations.csv')


# Creates df of relative bearings.  To be merged with a left join into train df.
bearing_df = pd.DataFrame.from_dict(bearing_dict, orient='index')  
bearing_labels = bearing_df.index.tolist()  
bearing_cols = []
for c in bearing_labels:
    label = c + '_bearing'
    bearing_cols.append(label)
bearing_df.columns = [bearing_cols]
bearing_df['Trap'] = bearing_df.index
# bearing_df.to_csv('../data/relative_bearings.csv')

In [53]:
train = pd.merge(train, dist_df, how='left', left_on='Trap', right_on='Trap')
train = pd.merge(train, bearing_df, how='left', left_on='Trap', right_on='Trap')

In [57]:
train.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,...,T161_bearing,T160_bearing,T001_bearing,T002_bearing,T003_bearing,T004_bearing,T005_bearing,T006_bearing,T007_bearing,T237_bearing
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,...,338.334690,301.684518,271.154623,360.000000,253.509807,258.410250,304.504646,184.615570,210.338070,340.015038
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,...,338.334690,301.684518,271.154623,360.000000,253.509807,258.410250,304.504646,184.615570,210.338070,340.015038
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,...,345.766428,326.802670,327.565913,30.316860,344.286932,334.280074,5.703600,77.178457,360.000000,347.906466
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,...,336.428294,305.032919,286.823557,317.610162,281.176010,281.329113,311.618042,231.808650,243.162832,337.764079
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,...,336.428294,305.032919,286.823557,317.610162,281.176010,281.329113,311.618042,231.808650,243.162832,337.764079


In [59]:
n_weather.head().T

,0,1,2,3,4
Station,1,2,1,2,1
Date,2007-05-07 00:00:00,2007-05-07 00:00:00,2007-05-08 00:00:00,2007-05-08 00:00:00,2007-05-09 00:00:00
Tmax,83,84,82,80,77
Tmin,47,50,54,60,61
Tavg,65,67,68,70,69
DewPoint,41,39,58,57,59
WetBulb,54,53,62,63,63
PrecipTotal,0.005,0.00,0.00,0.005,0.13
StnPressure,29.38,29.44,29.29,29.36,29.21
SeaLevel,30.12,30.12,30.03,30.02,29.94


In [60]:
train = pd.merge(train,n_weather,left_on=['Date','Weather_Station'],right_on=['Date','Station'])

In [63]:
train.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,...,TS_date-6,SH_date-6,RA_date-6,\+FC_date-6,SA_date-6,SG_date-6,PO_date-6,PL_date-6,SN_date-6,_date-1_date-6
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,...,0,0,0,0,0,0,0,0,0,2007-05-22
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,...,0,0,0,0,0,0,0,0,0,2007-05-22
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,...,0,0,0,0,0,0,0,0,0,2007-05-22
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,...,0,0,0,0,0,0,0,0,0,2007-05-22
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,...,0,0,0,0,0,0,0,0,0,2007-05-22


In [64]:
train.to_csv('../data/train.csv',index=False)